# EDA Application with a Real Data Set - Draft Version 1
## https://www.kaggle.com/c/santander-customer-satisfaction

## Data Set

- Santander Bank is asking to help identify dissatisfied customers in their relationship with hundreds of anonymized features. Please visit the website and read details about it.

- 743 columns (ID, TARGET, and features), 76020 rows: train and test data sets.

- The "TARGET" column is the variable to predict. It equals one for unsatisfied customers and 0 for satisfied customers.

- Data file descriptions:

        train.csv - the training set including the target
        test.csv - the test set without the target
        sample_submission.csv - a sample submission file in the correct format


## Goal

- The task: 
    1. to describe the data set
        - data type for each column 
        - missingness
        - univariate summaries
        - bivariate summaries
            - associations
            - correlations
            - patterns
        - multivariate summaries
        - more
    2. to predict the probability that each customer in the test set is an unsatisfied customer.
    3. to predict if a customer is satisfied or dissatisfied with their banking experience.


## Housekeeping

Know: 
- where you are working
- version check
- shorcuts: https://youtu.be/d0oBRIONOEw

Basic packs to use:
- numpy
- pandas
- seaborn
- matplotlib

In [ ]:
# Directory working
%pwd #look at the current work dir. import os
#%cd

### Step 1: Source and load the data

In [ ]:
# Import necessary libraries and load the data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Check version
import pandas as pd
pd.__version__

In [ ]:
# Baurjan's importing code
# Skip
#import boto3
#client = boto3.client('s3') #amazon web service usage -- with s3 -- boto3 is library. Run this in SageMake. Next Workshop.
#train_path = 's3://classdataset/train.csv'
#test_path = 's3://classdataset/test.csv'
#train= pd.read_csv(train_path)
#test = pd.read_csv(test_path)

In [ ]:
#import data from directory
train = pd.read_csv("santander-customer-satisfaction/train.csv", index_col='ID') # the train dataset is now a Pandas DataFrame
test = pd.read_csv("santander-customer-satisfaction/test.csv", index_col='ID') # the train dataset is now a Pandas DataFrame
#train = pd.read_csv("Documents/GitHub/ML/santander-customer-satisfaction/train.csv", index_col='ID') # the train dataset is now a Pandas DataFrame
#test = pd.read_csv("Documents/GitHub/ML/santander-customer-satisfaction/test.csv", index_col='ID') # the train dataset is now a Pandas DataFrame

In [ ]:
# Alternative way if data is undet the same folder
train = pd.read_csv("train.csv")                    

### Step 2: Data Wrangling 

In [ ]:
# We will use train data set for EDA
# Look the number of rows and columns
train.shape

In [ ]:
# Data types
train.dtypes

In [ ]:
# Check out the data types, number of entries and other details
train.info()

In [ ]:
# Check out the first 5 rows of the data set
train.head()

In [ ]:
# Given we couldn't see NaNs from the df.info(), let's do this another way
train.isnull().sum() #.nlargest() #.sum() 

In [ ]:
# Number of columns with all zeros as average
np.sum(train.mean()==0)

In [ ]:
# Which columns have zeros?
?

In [ ]:
# Columns with all zeros
# This code is NOT recommended b/c mean=0 doesn't mean all values are ZERO
zero_mean = (train.mean()==0).to_frame(name='ZeroMean')

In [ ]:
zero_mean = zero_mean.loc[zero_mean.ZeroMean]
print(zero_mean)

In [ ]:
# We can get rid of the columns with all zeros, there is no information to gain from them
# This is dangerous way to do!
remove_features = list(zero_mean.index)
print(remove_features)

In [ ]:
train = train.drop(remove_features, axis=1)
train.shape

Notice the number of columns that decreased from original 371 to 337

In [ ]:
# Number of columns with all zeros
np.sum(train.mean()==0)

In [ ]:
# Uniqueness of values using train.unique for each column
# Effective and automatic way?
train['TARGET'].unique()

In [ ]:
# Let's look at classes of the target 
train[['TARGET', 'var3']]

In [ ]:
# Percent of happy-unhappy customers
train['TARGET'].value_counts() / train.shape[0]

# See this data is a unbalanced data. When modeling, this produces an issue tht needs to be handled. 

In [ ]:
# Happy customers: TARGET==0 vs Unhappy custormers: TARGET==1
df = pd.DataFrame(train.TARGET.value_counts())
df['Percentage'] = 100*df['TARGET']/train.shape[0]
df

### Discover data related issues, type issues, all issues before running EDA

In [ ]:
# Check the range of each feature and target values
train.describe()

### Write a code to get all the ranges of the data set

In [ ]:
ab = []

for i in train.columns:
    ab.append([i, train[i].min(),train[i].max(), train[i].max()-train[i].min()])

print(ab) #see this is not good way to print!

#? Research: Effect of Kurtosis and Skewness on Scaling Method (Feature Engineering)
#? Shape of Distri
#? Use of Skewness in Modeling. Next Topic.

In [ ]:
# Give column names and make data frame of pandas, then print
cols = ['cols','min','max', 'range'] #add more statistic
df_ranges = pd.DataFrame(ab,columns=cols)
df_ranges

In [ ]:
# When scaling, high values will skew the transformation. So use percentile or robust scaling method
# How to handle these high values or coded values in a variable? -999999
?

In [ ]:
df_ranges.sort_values(by=['range'], ascending=False, axis=0)

In [ ]:
# See one -999999 in var3
train.var3.value_counts() #all
train.var3.value_counts()[:10] #first 10 levels

In [ ]:
# Dig into issues and redefine
train.loc[train.var3==-999999].shape

In [ ]:
# Replace -999999 with 0
train = train.replace(-999999,0)
train.loc[train.var3==-999999].shape

In [ ]:
# Create a new feature that points this unusual values
?

### Task. Your observations and data wrangling to discover issues
Please play with the data set. Use basic pandas codes and observe and explore issues. 

### Step 3: Exploratory Data Analysis (EDA) with Basic Functions

In [ ]:
train.describe()

In [ ]:
# Histogram of Each Feature
# This will generate bar charts/histogram for each. Later, after feature reductions, you can use it again.
class color:
   BOLD = '\033[1m'
   END = '\033[0m'
for i, col in enumerate(train.columns):
    sns.set()
    plt.figure(figsize=(5,3)) 
    plt.hist(train[col], bins=30)
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.title(col)
    plt.tight_layout()
    plt.show()

In [ ]:
# Calculate correlation matrix
df_corr = train.corr()
print(df_corr)

In [ ]:
# Task. Sort, get highest correlations (+-) using abs(), unstack(), sort_values(kind="quicksort") and Sprint

In [ ]:
# Lets pick some variables and get correlation
train[['var36', 'num_var5', 'var38', 'TARGET']].corr()

### Seaborn

In [ ]:
# Heatmap using seaborn
# You may want to run this after eliminating features
import seaborn as sns
sns.heatmap(train[['var36', 'num_var5', 'var38', 'TARGET']].corr(),annot=True)

In [ ]:
# Scatter plot with fitted line
sns.lmplot(x='var38', y='num_var5', data=train)
plt.show()

# Evaluate the plot. Discuss does this tell us anythings?

In [ ]:
sns.lmplot(x='var38', y='num_var5', hue='TARGET', data=train, fit_reg=True)
plt.show()

In [ ]:
# Boxplot
sns.boxplot(data=train[['var36','var38','num_var5', 'TARGET']])

plt.show()

# Discuss: On the same scale plot of fatures, it is not good idea. Need to do seperate or scale.
# Task: How to scale and how to seperately plot boxplots?

In [ ]:
sns.violinplot(data=train[['var38']])
plt.show()

In [ ]:
# Change figure size
plt.figure(figsize=(14,5))

sns.violinplot(x='TARGET', y='var38', data=train)

plt.show()

In [ ]:
# Distribution plot
sns.distplot(train['var36'], color='blue')

plt.show()

In [ ]:
# Count plot
sns.countplot(x='var36', data=train)

# Rotate labels
plt.xticks(rotation=90)

In [ ]:
# Joint plot: bivariate
# This take time
sns.jointplot(train.var38, train.num_var5, data=train, kind='kde', color='blue')

### Plotly

It is interactive effective tool for data visualization and EDA

In [ ]:
import plotly.express as px

In [ ]:
# Scatter plot
fig = px.scatter(train, x='var38', y='num_var5')

fig.show()
# This may not work right away. May need to install pack from pip or terminal. Jupyter-lab has issues with it.

In [ ]:
# Hovering features
fig = px.scatter(train, x='var38', y='num_var5', hover_data=['TARGET'])

fig.show()

In [ ]:
# Showing total along with hovering features
# Fix it
fig = px.scatter(train, x='var36', y='num_var5', hover_data=['TARGET'], size='var38', color='blue')

fig.show()

In [ ]:
# Showing total along with hovering features
fig = px.box(train[['var36','var38']],  color='blue')

fig.show()

In [ ]:
# Task: get correlation matrix as corr_matrix, list it, label it by labels=list(corr_matrix), use px.imshow(corr_matrix,x=labels, y=labels) and fig.show()
# px.histogram(), px.density_heatmap()

In [ ]:
# Task: use of list(), range(), list(rage(train.shape[0])), .sort_values()

## Usefel codes

In [ ]:
# df is data set: Then

# if you want to get the rows where the color is green, then you’ll need to apply:
df.loc[df[‘Color’] == ‘Green’]

# Green and Rectangle
df.loc[(df[‘Color’] == ‘Green’) & (df[‘Shape’] == ‘Rectangle’)]

# Use of OR
df.loc[(df[‘Color’] == ‘Green’) | (df[‘Shape’] == ‘Rectangle’)]

#To get all the rows where the price is equal or greater than 10, you’ll need to apply this condition:
df.loc[df[‘Price’] >= 10]

#You can use the combination of symbols != to select the rows where the price is not equal to 15:
df.loc[df[‘Price’] != 15]


### Step 3b: Issues with This Type of Data

- Too many zeros. How to handle?

- How to detect associations among any two combination? (EDA with no Target) 

- Association with Target Variable (so feature selection can be employed)

- When to Use and Why Association?
    - To measure association (chi-sq, RR, OR, conditional proportion etc.) between two categorical variables

- When to Use and Why Correlation?
    - To measure between two numerical linear relationships
    

- When to not Use?

- Alternatives?


### Search: Why Mr. B write this? Make critiques

Notice how we can't even see the correlation matrix for all of the variables due to large number of columns. We will try the following methods:

    1. Melt the data that will result into decrease in number of columns but increase in number of rows. 
    
    2. Shrink the size of the columns to the most important 10 features by using Univariate feature selection technique.

### Step 3c: Feature Importance and Selection

This is just one application. Many methods and approach exist. The instructor will briefly go over the file, 'Feature Engineering, Data Reduction, and Curse.doc'

In general, the choice of evaluation metric heavily influences the algorithm, and it is these evaluation metrics which distinguish between the three main categories of feature selection algorithms: 

- wrappers, 
- filters and 
- embedded methods.

Filters are most common and cheap.

In [ ]:
# Let's scale the features first before feature elimination: Why?
from sklearn.preprocessing import MinMaxScaler
x = train.drop(['TARGET'], axis=1)
y = train['TARGET']
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)
# Task: Use other scaling method suited for sparse data. See 'Scaling, Transformation, Metrics, Curse.doc'

In [ ]:
# Apply SelectKBest class to extract the 10 features that best explains the association relationships between Target and Univariate feature (chi-squa results are ranked) 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2 #see other options https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html
bestfeatures = SelectKBest(score_func=chi2, k=50)
fit = bestfeatures.fit(x_scaled, y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x.columns)
featureScores = pd.concat([dfcolumns, dfscores], axis=1)
featureScores.columns = ['Feature', 'Score']
print(featureScores.nlargest(50, 'Score'))

In [ ]:
columns_10best_chi = ['num_meses_var5_ult3', 'ind_var5', 'ind_var30', 
           'var36', 'ind_var8_0', 'ind_var13', 
           'ind_var13_0', 'ind_var12_0', 
          'num_var5', 'ind_var13_corto', 'TARGET']

In [ ]:
# Now, we have 11+1 features from train set to proceed
sub_train = train[columns_10best_chi]

In [ ]:
# And take the same features from test set
columns_10best_chi = ['num_meses_var5_ult3', 'ind_var5', 'ind_var30', 
           'var36', 'ind_var8_0', 'ind_var13', 
           'ind_var13_0', 'ind_var12_0', 
          'num_var5', 'ind_var13_corto']
sub_test = test[columns_10best_chi]

In [ ]:
print(sub_train.shape, sub_test.shape)

In [ ]:
sub_train.dtypes

## Feature Elimination Techniques

Univariate Feature Selection from Scikit

    2.1. GenericUnivarietSelect
    2.2. SelectKBest
    2.3. SelectPercentile
    2.4. SelectFpr
    2.5. SelectFdr
    2.6. SelectFwe
    2.7. chi2
    2.8. f_classif
    2.9. f_regerssion
    2.10. mutual_info_classif
    2.11. mutual_info_regression
    
Also usefel methods below. Plase play wtih these codes. May need fixing:

0) Removing Constant Features
1) Removing Quasi-constant: Quasi-constant features are those that show the same value for the great majority of the observations of the dataset.
2) Removing Feature From Low Variance

In [ ]:
# 0) Removing Constant Features
from sklearn.feature_selection import VarianceThreshold

# reload the original data, remove -99999, and then run this: see how zeros are removes easily, better that mean.zero method.
# fit finds the features with zero variance
sel = VarianceThreshold(threshold=0)
sel.fit(train)
X1 = train.drop(columns=train.columns[~sel.get_support()])
#X1t = test.drop(columns=test.columns[~sel.get_support()])

print(X1.shape)
#print(X1t.shape)

print(f'New sahape {X1.shape} number of feature removed {train.shape[1] - X1.shape[1]}')

In [ ]:
# 1) Removing Quasi-constant: Quasi-constant features are those that show the same value for the great majority of the observations of the dataset.

sel_quasi = VarianceThreshold(threshold=0.01)  # 0.1 indicates 99% of observations approximately
sel_quasi.fit(X1)

X2 = X1.drop(columns=X1.columns[~sel_quasi.get_support()])
#X2t = X1t.drop(columns=X1t.columns[~sel_quasi.get_support()])

print(f'New sahape {X2.shape} number of feature removed {X1.shape[1] - X2.shape[1]}')

In [ ]:
result = pd.DataFrame(sel_quasi.fit_transform(train))
print(result)

In [ ]:
# View amount of variances
print(sel_quasi.variances_)

In [ ]:
# Find the remaining column id
remain_features_id = np.where(sel_quasi.variances_ > sel.threshold)

# Assign remaining column name
result.columns = train.columns[remain_features_id]

print(result)

In [ ]:
# 2) Removing Feature From Low Variance
# Briefly in a toy example. Some codes are useful to apply to the data set.
#VarianceThreshold is a simple baseline approach to feature selection. It removes all features whose variance doesn’t meet some threshold. By default, it removes all zero-variance features, i.e. features that have the same value in all samples.
#As an example, suppose that we have a dataset with boolean features, and we want to remove all features that are either one or zero (on or off) in more than 80% of the samples. Boolean features are Bernoulli random variables, and the variance of such variables is given by
#so we can select using the threshold .8 * (1 - .8):

from sklearn.feature_selection import VarianceThreshold

# First Example 
X = [[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]]
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(X)

print(f'Variance value per columng {sel.variances_}, threshold {sel.threshold : 2.2f}')

sel = VarianceThreshold(threshold=0.01)
result = pd.DataFrame(sel.fit_transform(df_train))

sel.variances_

# Find the remaining column id
remain_features_id = np.where(sel.variances_ > sel.threshold)

# Assign remaining column name
result.columns = df_train.columns[remain_features_id]

print(result)

In [ ]:
# Let's get best 20 features using constant, quasi variances, and Chi-squ best selection method
# Rerun the original data set, remove -99999 and run next:
sel_quasi = VarianceThreshold(threshold=0.01)  # 0.1 indicates 99% of observations approximately
sel_quasi.fit(X1)

train_v = train.drop(columns=X1.columns[~sel_quasi.get_support()])

print(f'New sahape {train_v.shape} number of feature removed {train.shape[1] - train_v.shape[1]}')

In [ ]:
# Without applying scaling: run next. Then Use scaling and rerun. See what changes
# Apply SelectKBest class to extract the 10 features that best explains the association relationships between Target and Univariate feature (chi-squa results are ranked) 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2 #see other options https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html

x = train_v.drop(['TARGET'], axis=1)
y = train_v['TARGET']

bestfeatures = SelectKBest(score_func=chi2, k=50)
fit = bestfeatures.fit(x, y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x.columns)
featureScores = pd.concat([dfcolumns, dfscores], axis=1)
featureScores.columns = ['Feature', 'Score']
print(featureScores.nlargest(50, 'Score'))

## Discussion
Are all of these categorical variables? is it a problem when building predictive modeling?

## Need Powerful Tools and Packs

### Package 1: Pandas-Profiling

Import the pack or upoad it from https://github.com/pandas-profiling/pandas-profiling.

Shortly, this generates profile reports from a pandas DataFrame. The pandas df.describe() function is great but a little basic for serious exploratory data analysis. pandas_profiling extends the pandas DataFrame with df.profile_report() for quick data analysis.


In [ ]:
# Install here or using pip: >> pip install pandas-profiling
from pandas_profiling import ProfileReport

In [ ]:
# No need if not running
sub_train.profile_report(style={‘full_width’:True})

In [ ]:
profile = ProfileReport(sub_train, title="Pandas Profiling Report")
print(profile)

In [ ]:
profile.to_widgets()

In [ ]:
profile.to_file("your_report.html")

__NOTE__: Pandas-profiling may NOT process the data due to its size and not enough compute resource

### Package 2:  ClfAutoEDA

- Read the article https://medium.com/analytics-vidhya/automated-eda-for-classification-77c25b847e43
- Download the py code in the directory you are working: https://github.com/jatinkataria94/EDA-Classification/blob/master/ClfAutoEDA.py

In [ ]:
# import the autoEDA module
from ClfAutoEDA import *

In [ ]:
sub_train.shape

In [ ]:
#Just play with small data portion
#Setting parameter values
target_variable_name='TARGET'
labels=['Unhappy','Happy']
#Calling EDA function with parameters of choice
df_processed,num_features,cat_features=EDA(df=sub_train,labels=labels,
                                         target_variable_name=target_variable_name,
                                         data_summary_figsize=(6,6),
                                         corr_matrix_figsize=(6,6), 
                                         corr_matrix_annot=True,
                                         pairplt=True)

__NOTE__: Pandas-profiling couldn't process the data due to its size and not enough compute resource

## Pack 3. Sweetviz

Read the article https://towardsdatascience.com/powerful-eda-exploratory-data-analysis-in-just-two-lines-of-code-using-sweetviz-6c943d32f34

Install it using pip install sweetviz and run the next.

In [ ]:
import sweetviz

In [ ]:
#This will tke too much time if 
my_report = sweetviz.compare([sub_train, "Train"], [sub_test, "Test"], "TARGET")

In [ ]:
my_report.show_html("Report.html") # Not providing a filename will default to SWEETVIZ_REPORT.html

## Pack 4. Pycaret

A complex level pack that does all!

Read the article https://github.com/pycaret/pycaret and examples https://github.com/pycaret/pycaret/tree/master/examples.

Install it using pip install pycaret.

Practice it with the dataset here.

# More Resources to Practice
- https://www.kaggle.com/c/santander-customer-satisfaction and see many notebooks prepared
- https://www.kaggle.com/cast42/exploring-features ==> nice notebook to practice
